In [104]:
import numpy as np
import hp2np 
import hexalate
import decam2hp
import jsonMaker
from os import getenv
import pandas as pd
import sys
import  matplotlib.pyplot as plt
import mags
from pyslalib import slalib

In [105]:
import datetime

#get a readable date/time from mjd
def mjd_to_date_time(mjd, with_date = False):
    mjd_integer = int(mjd)
    mjd_decimal = mjd - mjd_integer

    # Convert MJD integer to date
    reference_date = datetime.datetime(1858, 11, 17, 12, 0, 0)  # Reference MJD date
    target_date = reference_date + datetime.timedelta(days=mjd_integer)

    # Convert MJD decimal to time
    total_seconds = mjd_decimal * 24 * 60 * 60
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    time_str = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"
    
    if with_date:
        return target_date.strftime("%Y-%m-%d") + " " + time_str
    else:
        return time_str

In [106]:
inner_dec = np.random.uniform(0, 30, size=(20))
inner_ra = np.random.uniform(200, 340, size=(20))
inner_prob =np.random.uniform(0.5, 1.0, size=(20))
outer_ra = np.random.uniform(200, 340, size=(20))
outer_prob =np.random.uniform(0.0, 0.5, size=(20))

outer_dec = np.random.uniform(0, 30, size=(20))


In [107]:
inner_ra = np.array(inner_ra); inner_dec = np.array(inner_dec); 
outer_ra = np.array(outer_ra); outer_dec = np.array(outer_dec); 
inner_prob = np.array(inner_prob); outer_prob = np.array(outer_prob); 

In [108]:
expTime_inner = 90
expTime_outer = 90

In [109]:
expTime = [np.ones(inner_ra.size)*expTime_inner]
flt = 'r'
mjd = 60068

In [110]:
def get_hexinfo(ra, dec, prob, expTime, filt, mjd, best=False, camera="decam"):
    """
    Gets information about given hexes. Based on Jim's simplicity.calc
    code.

    Parameters:
    -----------
    ra: np.vector
        vector of hex ra
    dec: np.vector
        vector of hex dec
    prob: np.vector
        vector of hex summed prob from LIGO map
    expTime: np.vector
        vector of exposure times
    filt: str
        telescope filt.
    mjd: float
        modified julian date.
    best: bool
        True/False if use best or not
    camera: str
        telescope camera name to take in account.
        
    Outputs:
    --------
    hexinfo: pd.dataframe
        dataframe with each row representing a hex. contains ra, dec,
        prob, and rise and set times in MJD (minutes), and is sorted by 
        probability.
    """

    # find the highest prob hex
    best_ix = np.argmax(prob)

    # find night statistics
    night, sunset, sunrise = mags.findNightDuration(mjd, camera)
    night = np.float64(night)*24.
    sunset = np.float64(sunset)
    sunrise = np.float64(sunrise)

    # work every minute from sunset to sunrise
    mjd_list = np.arange(sunset, sunrise+1./1440., 1./1440.)
    night_mjds = np.arange(sunset, sunrise + (expTime / 86400), expTime / 86400)
#     night_mjds = mjd_list
    # calculate the limiting magnitudes
    limit_mag = []
    moon_sep = []
    moon_phase = []
    obs = mags.observed(ra,dec,prob, sunset, doMaps=False, verbose=False)
    for mjd in mjd_list :
        obs.resetTime(mjd)
        obs.limitMag(filt,exposure=expTime)
        limit_mag.append(obs.maglim)
    limit_mag = np.vstack(limit_mag)
    ix = limit_mag < 0; limit_mag[ix] = 0

    
    set_list = [ ]
    rise_list = [ ]

#     print(limit_mag)
    for i in range(len(limit_mag)):
        for j in range(len(limit_mag[i])):
            if limit_mag[i][j] !=0 and limit_mag[i-1][j] == 0:
                rise_list.append([mjd_list[i], ra[j], dec[j]])
            elif limit_mag[i-1][j] !=0 and limit_mag[i][j] == 0:
                set_list.append([mjd_list[i], ra[j], dec[j]])
                #print(f'setting j={j}, i={i}, {ra[j]}, {mjd_list[i]}')
    
    set_times = []
    rise_times = []
    rise_moonseps = [] 
    set_moonseps = [] 
    
    
    for i in range(len(ra)):
        sets =[]
        rises=[]
        
        #find rise and set times corresponding to each ra and dec
        for s in set_list:
            if s[1]==ra[i] and s[2]==dec[i]:
                sets.append(s[0])

        #find rise time for each ra and dec set (in other words, each hex)
        for r in rise_list:
            if r[1]==ra[i] and r[2]==dec[i]:
                rises.append(r[0])

        if len(sets) != 0:
            set_times.append(np.max(sets))
        elif len(sets) == 0:
            set_times.append(mjd_list[-1])
        if len(rises) !=0:
            rise_times.append(np.min(rises))
        elif len(rises) == 0:
            rise_times.append(mjd_list[0])

    rise_est = [mjd_to_date_time(time) for time in rise_times]
    set_est = [mjd_to_date_time(time) for time in set_times]

    return ra, dec, prob, rise_est, set_est, rise_times, set_times, night_mjds

In [111]:
ra, dec, prob, rise_est, set_est, rise_mjd, set_mjd, night_mjds = get_hexinfo(inner_ra, inner_dec, inner_prob, 90, flt, mjd)

False
Loading dust map: .//data/plank-ebv-HFI_CompMap_ThermaDustModel.fits
loading inverse stellar density = probability of recognition map
	 ... the sun is up
	 ... the sun is up


In [112]:
get_hexinfo(inner_ra, inner_dec, inner_prob, 90, flt, mjd)

False
Loading dust map: .//data/plank-ebv-HFI_CompMap_ThermaDustModel.fits
loading inverse stellar density = probability of recognition map
	 ... the sun is up
	 ... the sun is up


(array([304.04734449, 334.43722136, 279.43881489, 339.15302037,
        329.90210741, 270.52847627, 205.51647989, 295.26937254,
        215.39606383, 282.9219067 , 291.22773263, 319.37877183,
        260.13852175, 287.36367777, 294.61073173, 224.29777596,
        323.08123271, 237.93473066, 221.95331409, 252.95917931]),
 array([11.05694572, 20.21694438, 12.8297375 ,  4.70656287, 27.90375234,
        11.29203695,  1.80915825, 16.74972271,  4.18531552, 13.69032787,
         7.80251616, 12.29113713, 14.29828712, 25.63811515,  3.33683812,
         9.93818974,  1.2832931 ,  5.63869862, 23.32873022, 15.05534329]),
 array([0.52308636, 0.79992003, 0.55529345, 0.91196445, 0.64913663,
        0.82496287, 0.80853571, 0.96585062, 0.99818151, 0.82492851,
        0.89057089, 0.64729528, 0.66705689, 0.70654846, 0.55144578,
        0.93710711, 0.90684809, 0.75988399, 0.68144685, 0.59569549]),
 ['06:30:00',
  '09:07:00',
  '04:58:00',
  '08:30:00',
  '09:31:00',
  '04:17:00',
  '23:29:00',
  '06:16:00'

In [113]:
ra_out, dec_out, prob_out, rise_est_out, set_est_out, rise_mjd_out, set_mjd_out, night_mjds = get_hexinfo(outer_ra, outer_dec, outer_prob, 90, flt, mjd)

False
Loading dust map: .//data/plank-ebv-HFI_CompMap_ThermaDustModel.fits
loading inverse stellar density = probability of recognition map
	 ... the sun is up
	 ... the sun is up


In [114]:
print(rise_mjd[0])
CTIO_LAT = -30.16527778
CTIO_LON = -70.8125
CTIO_HEIGHT = 2215.
KPNO_LAT = 31.9600784
KPNO_LON = -111.598169
KPNO_HEIGHT = 2067.
MAUNAKEA_LAT = 19.826667
MAUNAKEA_LON = -155.471667
MAUNAKEA_HEIGHT = 4215.

60068.27083333437


In [115]:
from enum import Enum

class CameraType(Enum):
    DECam = "decam"
    DESI = "desi"
    HSC = "hsc"

In [116]:
CameraType("decam")

<CameraType.DECam: 'decam'>

In [117]:
class hex_object:
    """
    Adapted heavily from Jim's mags.py code (at least for calculations of lunar separation). 

    Internally all coordinates, time and space, are in radians, except MJD, but outputs are in degrees.

    Attributes:
    -----------
    ra: float
        Right Accesion of hex. 
    dec: float
        Declination of hex.
    prob: float
        Given hex's probability.
    rise_est: string
        EST and date of hex's rise time.
    set_est: string
        EST and date of hex's set time. 
    rise_mjd: float
        Modified Julian Date of hex rise time.
    set_mjd: float
        Modified Julian Date of hex set time.
    camera: str (Default: "decam")
        Camera type for observations. 
    """
    def __init__(self, mjd_date, ra, dec, prob, rise_est, set_est, rise_mjd, set_mjd, index = None, camera="decam"):
        self.prob = prob
        self.rise_est = rise_est
        self.set_est = set_est
        self.rise_mjd = rise_mjd
        self.set_mjd = set_mjd

        self.degToRad = 2. * np.pi / 360.
        self.radToDeg = 360 / 2 / np.pi

        self.observatory = CameraType(camera)

        self.lat = self.obs_lat * self.degToRad
        self.lon = self.obs_lon * self.degToRad
        self.height = self.obs_height

        self.ra = ra * self.degToRad
        self.dec = dec * self.degToRad
        
        self.mjd_date = mjd_date
        
        self.has_risen = 0
        
        if mjd_date >= self.rise_mjd:
            self.has_risen = 1

        self.calculate_lunar_separations()
        self.get_awesomeness_factor()
        
        self.index = index

#         print(self.rise_moon_sep, self.set_moon_sep)
        
    def get_awesomeness_factor(self):
        #self.awesomeness_factor = np.random.uniform(0, 1.0)*self.has_risen
        self.awesomeness_factor = (self.ra/10)*self.has_risen#np.random.uniform(0, 1.0)*self.has_risen
    def calculate_lunar_separations(self):
            self.resetTime(self.rise_mjd)
            rise_moon_sep = self.radToDeg * self.getLunarSeparation(self.ra, self.dec, self.moonData)

            self.resetTime(self.set_mjd)
            set_moon_sep = self.radToDeg * self.getLunarSeparation(self.ra, self.dec, self.moonData)

            self.rise_moon_sep = rise_moon_sep
            self.set_moon_sep = set_moon_sep

    def getLunarPosition(self, mjd, eastLongitude, latitude, lst):
        ra, dec, diam = slalib.sla_rdplan(mjd, 3, eastLongitude, latitude)
        ha = lst - ra
        zd = self.zenithDistance(ha, dec, latitude)
        return ra, dec, zd
    
    def getLunarSeparation(self, ra, dec, moonData):
        moon_ra, moon_dec = self.moonData[0], self.moonData[1]
        moon_sep = self.gc_separation(ra, dec, moon_ra, moon_dec)
        return moon_sep

    def obs_lat(self):
        if self.observatory == CameraType.DECam or self.observatory == CameraType.DESI:
            return CTIO_LAT
        elif self.observatory == CameraType.HSC:
            return MAUNAKEA_LAT

    def obs_lon(self):
        if self.observatory == CameraType.DECam or self.observatory == CameraType.DESI:
            return CTIO_LON
        elif self.observatory == CameraType.HSC:
            return MAUNAKEA_LON
        
    @property
    def obs_height(self):
        if self.observatory == CameraType.DECam or self.observatory == CameraType.DESI:
            return CTIO_HEIGHT
        elif self.observatory == CameraType.HSC:
            return MAUNAKEA_HEIGHT
        
    def mjdToLST(self, mjd, eastLongitude):
        gmst = slalib.sla_gmst(mjd)
        eqEquinoxes = slalib.sla_eqeqx(mjd)
        lst = gmst + eqEquinoxes + eastLongitude
        return lst
    def gc_separation(self, ra1, dec1, ra2, dec2):
        delDec = dec1-dec2
        delRa = ra1-ra2
        dhav = self.haversine(delDec)
        rhav = self.haversine(delRa)
        hav = dhav + np.cos(dec1)*np.cos(dec2)*rhav
        gc_distance = self.ahaversine(hav)
        return gc_distance

    def haversine(self, theta):
        hav = np.sin(theta/2.)**2
        return hav

    def ahaversine(self, x):
        ahav = 2*np.arcsin(np.sqrt(x))
        return ahav
    def equatorialToObservational(self, ra, dec, lst, latitude):
        try:
            if len(self.ra) > 1:
                ha = lst - ra
                zd = self.zenithDistance(ha, dec, latitude)
                ix = np.nonzero(ha > 180.*2*np.pi/360.)
                ha[ix] = ha[ix] - 360*2*np.pi/360.
                return ha, zd
        
        except:
            ha = lst - self.ra
            zd = self.zenithDistance(ha, dec, latitude)
            return ha, zd
        
    def zenithDistance(self, ha, dec, latitude):
        degToRad = 2.*np.pi/360.
        sinAltRad = np.sin(latitude)*np.sin(dec) + \
            np.cos(latitude)*np.cos(dec)*np.cos(ha)
        altRad = np.arcsin(sinAltRad)
        zenithDist = 90*degToRad - altRad
        return zenithDist
    
    def resetTime(self, mjd):
        self.mjd = mjd
        self.lst = self.mjdToLST(self.mjd, self.lon)
        self.ha, self.zd = self.equatorialToObservational(
            self.ra, self.dec, self.lst, self.lat)
        self.moonData = self.getLunarPosition(
            self.mjd, self.lon, self.lat, self.lst)
        self.moonZD = self.moonData[2]
        self.moonSep = self.getLunarSeparation(
            self.ra, self.dec, self.moonData)
        self.moonRa = self.moonData[0]
        self.moonDec = self.moonData[1]
    
    def display_hexinfo(self):
        print(f'Index: {self.index}, RA: {self.ra:0.3}, DEC: {self.dec:0.3}, PROB: {self.prob:0.3}, AWESOMENESS: {self.awesomeness_factor:0.3}')

In [118]:
def order_hexes(hex_list):
    new_list = sorted(hex_list, key=lambda x: x.awesomeness_factor, reverse=True)
    return new_list

In [119]:
hexes = []  # List of hexes
obs_order = []  # List to store ordered hexes
observe_mjds = []
ra_list = list(ra) # Initialize ra list
dec_list = list(dec)
prob_list = list(prob)
rise_est_list = list(rise_est)
set_est_list = list(set_est)
rise_mjd_list = list(rise_mjd)
set_mjd_list = list(set_mjd)
ra_list_outer = list(ra_out) # Initialize ra list
dec_list_outer = list(dec_out)
prob_list_outer = list(prob_out)
rise_est_list_outer = list(rise_est_out)
set_est_list_outer = list(set_est_out)
rise_mjd_list_outer = list(rise_mjd_out)
set_mjd_list_outer = list(set_mjd_out)

hex_tuple_inner = [ra_list, dec_list, prob_list, rise_est_list, set_est_list,
                  rise_mjd_list, set_mjd_list]
hex_tuple_outer = [ra_list_outer, dec_list_outer, prob_list_outer, 
                   rise_est_list_outer, set_est_list_outer, rise_mjd_list_outer, 
                   set_mjd_list_outer]


def remove_list_element(hex_list_tuple, index_to_remove):
    for hex_list in hex_list_tuple:
        hex_list.pop(index_to_remove)
        
def sort_hexes(hex_list_tuple, night_mjd_index, hexes_list):
    for i in range(len(hex_list_tuple[0])):
        this_hex = hex_object(night_mjds[night_mjd_index], hex_list_tuple[0][i], hex_list_tuple[1][i], 
                             hex_list_tuple[2][i], hex_list_tuple[3][i], hex_list_tuple[4][i], 
                             hex_list_tuple[5][i], hex_list_tuple[6][i], index=i)
        hexes_list.append(this_hex)
        
    has_nonzero_awesomeness = any(hex_obj.awesomeness_factor != 0 for hex_obj in hexes_list)
        
    if has_nonzero_awesomeness:
        hexes_list = order_hexes(hexes_list)
        obs_order.append(hexes_list[0])
        index_to_remove = hexes_list[0].index
        observe_mjds.append(night_mjds[j])

        # Remove elements at the specified index from all lists
        remove_list_element(hex_list_tuple, index_to_remove)
        return True
            
    else: 
        return False
            
for j in range(len(night_mjds)):
    hexes = []
    hexes_outer = []
    if len(ra_list) != 0:
        found_hex = sort_hexes(hex_tuple_inner, j, hexes)

        if len(ra_list_outer) != 0 and not found_hex:
            found_outer_hex = sort_hexes(hex_tuple_outer, j, hexes_outer)

    elif len(ra_list_outer) != 0:
            found_outer_hex = sort_hexes(hex_tuple_outer, j, hexes_outer)

    else:
        for i in range(len(obs_order)):
            print(f'At {mjd_to_date_time(observe_mjds[i])} will observe this hex:')
            obs_order[i].display_hexinfo()
        break


At 23:30:00 will observe this hex:
Index: 6, RA: 3.59, DEC: 0.0316, PROB: 0.809, AWESOMENESS: 0.359
At 00:00:00 will observe this hex:
Index: 19, RA: 3.63, DEC: 0.163, PROB: 0.0501, AWESOMENESS: 0.363
At 00:15:00 will observe this hex:
Index: 7, RA: 3.76, DEC: 0.073, PROB: 0.998, AWESOMENESS: 0.376
At 00:43:30 will observe this hex:
Index: 18, RA: 3.9, DEC: 0.0364, PROB: 0.238, AWESOMENESS: 0.39
At 01:09:00 will observe this hex:
Index: 13, RA: 3.91, DEC: 0.173, PROB: 0.937, AWESOMENESS: 0.391
At 01:10:30 will observe this hex:
Index: 15, RA: 3.61, DEC: 0.46, PROB: 0.44, AWESOMENESS: 0.361
At 01:21:00 will observe this hex:
Index: 10, RA: 3.69, DEC: 0.442, PROB: 0.223, AWESOMENESS: 0.369
At 01:27:00 will observe this hex:
Index: 12, RA: 3.81, DEC: 0.361, PROB: 0.182, AWESOMENESS: 0.381
At 01:49:30 will observe this hex:
Index: 14, RA: 4.15, DEC: 0.0984, PROB: 0.76, AWESOMENESS: 0.415
At 01:54:00 will observe this hex:
Index: 14, RA: 3.87, DEC: 0.407, PROB: 0.681, AWESOMENESS: 0.387
At 

In [39]:
for j in range(len(night_mjds)):
    for i in range(len(ras)):
    this_hex = (night_mjds[j], ra[i], dec[i], prob[i], rise_est[i], set_est[i], rise_mjd[i], set_mjd[i])
    

IndentationError: expected an indented block (1559460977.py, line 3)

In [35]:
used_indices = []
# Populate hexes list
for j in range(len(night_mjds)):
    this_ra_list 
    hexes = []
    while True:
    for i in range(len(ra)):
        this_hex = hex_object(night_mjds[j], ra[i], dec[i], prob[i], rise_est[i], set_est[i], rise_mjd[i], set_mjd[i])
        hexes.append(this_hex)
        hexes.sort(key=lambda hex_obj: hex_obj.awesomeness_factor)

# Sort hexes based on awesomeness_factor in descending order
# hexes.sort(key=lambda hex_obj: hex_obj.awesomeness_factor, reverse=True)

obs_order = []  # List to store ordered hexes

# for i in range(len(hexes)):
#     obs_order.append(hexes[i])
#     rtod = 360 / (2 * np.pi)


In [ ]:
d = {'ra': [rtod*thishex.ra for thishex in hexes], 'dec': [rtod*thishex.dec for thishex in hexes], 'awesomeness factor': [thishex.awesomeness_factor for thishex in hexes]}

In [ ]:
d2 = {'ra': [rtod*thishex.ra for thishex in new_list], 'dec': [rtod*thishex.dec for thishex in new_list], 'awesomeness factor': [thishex.awesomeness_factor for thishex in new_list]}

In [ ]:
df = pd.DataFrame(data=d)
df2 = pd.DataFrame(data=d2)

In [ ]:
print(df)

In [ ]:
print(df2)